Imports for this project

In [2]:
import numpy as np
import os 
from PIL import Image
import matplotlib.image as img

Run the following code to parse the dataset ONLY ONCE - it takes a long time

In [17]:
#----------------------------------SETTING UP THE DATA---------------------------------------------------------
#----------------------------------Training Data----------------------------------------------------------------

#feature vector size
X = np.zeros((375000, 2200))
#open the url and read from the lines
url_name = "/Users/benjaminadame/downloads/matched_train.txt"
lines = open(url_name).readlines()
#get rid of the firt new line character
lines.pop(0)
#iteration to format the names and the matched pictures
for i,line in enumerate(lines):

    tokens = line.split("\t")
    tokens[2] = tokens[2].replace("\n", "")
    url_name1 = "/Users/benjaminadame/Downloads/lfw-deepfunneled/" + tokens[0] + "/" + tokens[0] + "_" + "{:04d}".format(int(tokens[1]))+".jpg"
    url_name2 = "/Users/benjaminadame/Downloads/lfw-deepfunneled/" + tokens[0] + "/" + tokens[0] + "_" + "{:04d}".format(int(tokens[2]))+".jpg"
    image1 = img.imread(url_name1)
    image2 = img.imread(url_name2)
    #reshape the images--- one image remains in one column
    image1 = image1.reshape(250*250*3,)
    image2 = image2.reshape(250*250*3,)

    #input images into a feature vector in this case it is X
    #0:187500 is accounting for the rows in the feature vector the same can be said for 187500:37500
    # i is just the number of columns which represents the number of images that are being used
    X[0:187500,i] = image1
    X[187500:375000,i] = image2

In [18]:
url_name_mis = "/Users/benjaminadame/downloads/mismatched_train.txt"
lines2 = open(url_name_mis).readlines()

#iteration to format the names and the mismatched pictures
for i,line in enumerate(lines2):

    tokens = line.split("\t")
    tokens[2] = tokens[2].replace("\n", "")
    url_name1 = "/Users/benjaminadame/Downloads/lfw-deepfunneled/" + tokens[0] + "/" + tokens[0] + "_" + "{:04d}".format(int(tokens[1]))+".jpg"
    url_name2 = "/Users/benjaminadame/Downloads/lfw-deepfunneled/" + tokens[2] + "/" + tokens[2] + "_" + "{:04d}".format(int(tokens[3]))+".jpg"
    image1 = img.imread(url_name1)
    image2 = img.imread(url_name2)
    #reshape the images--- one image remains in one column
    image1 = image1.reshape(250*250*3,)
    image2 = image2.reshape(250*250*3,)
    #input images into a feature vector in this case it is X
    #0:187500 is accounting for the rows in the feature vector the same can be said for 187500:37500
    # i is just the number of columns which represents the number of images that are being used
    X[0:187500,1100+i] = image1
    X[187500:375000,1100+i] = image2

In [19]:
#creating the labels for the training set 
Y = np.zeros((1, 2200))
for i in range(1100):
    Y[0, i] = 1

In [20]:
np.save('X_train', X)
np.save('Y_train', Y)

In [ ]:
#--------------------------------------------TESTING DATA---------------------------------------------
#feature vector size
X_test = np.zeros((375000, 1000))
#open the url and read from the lines
url_name = "/Users/benjaminadame/downloads/matched_test.txt"
lines = open(url_name).readlines()
#get rid of the firt new line character
lines.pop(0)
#iteration to format the names and the matched pictures
for i,line in enumerate(lines):

    tokens = line.split("\t")
    tokens[2] = tokens[2].replace("\n", "")
    url_name1 = "/Users/benjaminadame/Downloads/lfw-deepfunneled/" + tokens[0] + "/" + tokens[0] + "_" + "{:04d}".format(int(tokens[1]))+".jpg"
    url_name2 = "/Users/benjaminadame/Downloads/lfw-deepfunneled/" + tokens[0] + "/" + tokens[0] + "_" + "{:04d}".format(int(tokens[2]))+".jpg"
    image1 = img.imread(url_name1)
    image2 = img.imread(url_name2)
    #reshape the images--- one image remains in one column
    image1 = image1.reshape(250*250*3,)
    image2 = image2.reshape(250*250*3,)

    #input images into a feature vector in this case it is X
    #0:187500 is accounting for the rows in the feature vector the same can be said for 187500:37500
    # i is just the number of columns which represents the number of images that are being used
    X_test[0:187500,i] = image1
    X_test[187500:375000,i] = image2

In [41]:
url_name_mis = "/Users/benjaminadame/downloads/mismatched_test.txt"
lines2 = open(url_name_mis).readlines()

#iteration to format the names and the mismatched pictures
for i,line in enumerate(lines2):

    tokens = line.split("\t")
    tokens[2] = tokens[2].replace("\n", "")
    url_name1 = "/Users/benjaminadame/Downloads/lfw-deepfunneled/" + tokens[0] + "/" + tokens[0] + "_" + "{:04d}".format(int(tokens[1]))+".jpg"
    url_name2 = "/Users/benjaminadame/Downloads/lfw-deepfunneled/" + tokens[2] + "/" + tokens[2] + "_" + "{:04d}".format(int(tokens[3]))+".jpg"
    image1 = img.imread(url_name1)
    image2 = img.imread(url_name2)
    #reshape the images--- one image remains in one column
    image1 = image1.reshape(250*250*3,)
    image2 = image2.reshape(250*250*3,)
    #input images into a feature vector in this case it is X
    #0:187500 is accounting for the rows in the feature vector the same can be said for 187500:37500
    # i is just the number of columns which represents the number of images that are being used
    X_test[0:187500,500+i] = image1
    X_test[187500:375000,500+i] = image2

In [43]:
#creating the labels for the testing set

Y_test = np.zeros((1, 1000))
for i in range(500):
    Y_test[0, i] = 1

In [44]:
#saving the files 
np.save('X_test', X_test)
np.save('Y_test', Y_test)

Start Machine Learning Stuff from this cell onward

In [3]:
# Loading Files.....Loading takes 3 minutes.
X_train = np.load('X_train.npy')
Y_train = np.load('Y_train.npy')
X_test = np.load('X_test.npy')
Y_test = np.load('Y_test.npy')

In [4]:
#--------------------------------NETWORK ARCHITECTURE------------------------------------------
#Setting up X_train and Y_train

layer_dims = [X_train.shape[0],5,3,2,1]

#sigmoid function 
def sigmoid(z):
    return 1 / (1+np.exp(-z))

#loss function
def loss (A,Y):
    l = Y * np.log(A) + (1-Y) * np.log(1-A)
    return l

#cost function
def cost (m,A,Y):
    c = (-1/m) * np.sum(loss(A,Y))
    return c

#initialize w and b, ld is short for layer_dims 
def initialize_parameters(ld):
    W = []
    b = []
    
    for i in range (0,4):
        W.append(np.random.randn(ld[i+1],ld[i]))
        b.append(np.random.randn(ld[i+1],1))  
    return W,b


Foward Propagation 

In [5]:
#foward propagation 
def forward_prop(w,b,x,y,m):
    
    z = []
    a = []
    
    
    #propagate through each layer 
    #first layer 
    a.append(x)
    z.append(np.dot(w[0],x)+b[0])
    a.append(np.tanh(z[0]))
    for i in range(1,3):
        z.append(np.dot(w[i],a[i])+b[i])
        a.append(np.tanh(z[i]))
        
    #last layer 
    #w.append(np.random.randn(ld[4],ld[3]))
    #b.append(np.random.randn(ld[4],1))
    z.append(np.dot(w[3],a[3])+b[3])
    a.append(sigmoid(z[3]))
    
    #compute cost 
    J = cost(m,a[4],y)
    return a,J

#### Backward Propagation

In [6]:
#back propagation 
def backward_prop(a,y,m):
    #initialize the array structure 
    dz = [0,0,0,0]
    dw = [0,0,0,0]
    db = [0,0,0,0]
    
    #third layer initialization
    dz[3] = a[4] - y
    dw[3] = (1/m)*np.dot(dz[3],a[3].T)
    db[3] = (1/m)*np.sum(dz[3],axis=1,keepdims=True)
    for i in range (0,2):
        dz[2-i] = np.dot(dw[2-i+1].T,dz[2-i+1])*(np.ones(a[2-i+1].shape)-np.power(a[2-i+1],2))
        dw[2-i] = (1/m)*np.dot(dz[2-i],a[2-i].T)
        db[2-i] = (1/m)*np.sum(dz[2-i],axis=1,keepdims=True)
    return dw,db
    

Gradient Descent 

In [7]:
def grad(w,b,dw,db,lr):
    for i in range (0,4):
        w[i] = w[i] - (lr * dw[i])
        b[i] = b[i] - (lr * db[i])
        
    return w,b

Main program 

In [15]:
num_iterations = 30
learning_rate = 1
m = 2200
#initialize our parameters 
W,b = initialize_parameters(layer_dims)
for i in range (num_iterations):
    #propagate forward and back 
    A,J = forward_prop(W,b,X_train,Y_train,m)
    dw,db = backward_prop(A,Y_train,m)
    #apply the gradient descent 
    W,b = grad(W,b,dw,db,learning_rate)
    print(J)
    
    

0.748123992768185
0.7318868477854586
0.7265887320532277
0.7245364902082629
0.7234175860260902
0.7225693187566116
0.7218047525742017
0.7210716196716782
0.7203554797149418
0.7196524412491161
0.7189615179829665
0.7182825027673686
0.7176153841467957
0.7169601903293904
0.7163169496074918
0.7156856814710394
0.7150663952647797
0.7144590904054137
0.7138637567549686
0.7132803749017258
0.7127089163546341
0.7121493436920017
0.7116016106959249
0.7110656624913281
0.7105414356997292
0.710028858612655
0.7095278513866936
0.7090383262605525
0.7085601877935745
0.7080933331246437


Print cost-----> you need to have the cost decreasing every time 

Prediction

In [29]:
def predict(x,w,b,m):
    a = x
    y_pred = np.zeros((1,m))
    for i in range (0,3):
        a = np.tanh(np.dot(w[i],a)+b[i])
    a = sigmoid(np.dot(w[3], a)+b[3])
    for i in range(0,m):    
        if (a[0,i]<0.5):
            y_pred[0,i] = 0
        else:
            y_pred[0,i] = 1
        
    return a, y_pred



        
    

HOW ACCURATE WILL THIS TEST BE........?

In [30]:
a, y_pred = predict(X_test,W,b,1000)
diff = np.abs(y_pred-Y_test)
negative_acc = np.mean(diff)
acc = 1-negative_acc

print("TEST ACCURACY: " + str(acc*100)+"%")


TEST ACCURACY: 93.7%
